In [1]:
%reload_ext autoreload
%autoreload 2

%cd /Users/chompk.visai/Works/cdao/connext/connext-galxe-analytics/

/Users/chompk.visai/Works/cdao/connext/connext-galxe-analytics


In [2]:
import logging
from datetime import datetime, timedelta
from typing import Dict, List

import numpy as np
import pandas as pd
from dotenv import load_dotenv

from api.connext import ConnextAPI
from api.contract import SmartContract
from api.constant import Chain
from api.token import Token
load_dotenv(".env")

True

## Define Functions

In [3]:
logging.basicConfig(level=logging.INFO)


def unix_to_datetime(ts: int) -> datetime:
    return datetime.utcfromtimestamp(int(ts))
    

def get_abis():
    erc20_abi = Token.address_mapper[Chain.GNOSIS]["canonical"][Token.USDC].abi
    diamond_abi = ConnextAPI().scan_api[Chain.GNOSIS].diamond_contract.abi
    return erc20_abi + diamond_abi


def get_topic_resolver():
    diamond = ConnextAPI().scan_api[Chain.GNOSIS].diamond_contract
    
    topic2sig = {}
    for abi in get_abis():
        name = abi["name"]
        inputs = ",".join([_item["type"] for _item in abi["inputs"]])
        sig = f"{name}({inputs})"
        topic_id = diamond.provider.toHex(diamond.provider.keccak(text=sig))

        topic2sig[topic_id] = [name] + [_item["type"] for _item in abi["inputs"]]
    return topic2sig


def get_liquidity_txs(
    chains: List[Chain] = [Chain.POLYGON, Chain.ARBITRUM_ONE, Chain.BNB_CHAIN, Chain.GNOSIS, Chain.OPTIMISM],
    filter_topics: List[str] = ["Transfer"],
    filter_function: List[str] = ["addSwapLiquidity", "removeSwapLiquidity"],
    blacklist_token: List[str] = [],) -> pd.DataFrame:
    """Get txs involved with add/remove stable liquidity"""
    liquidity_txs = []
    topic2sig = get_topic_resolver()

    # iterate over chains that support liquidity providing
    for chain in chains:
        provider = SmartContract.get_default_provider(chain)
        # iterate over txn
        for tx in data[chain]:
            # skip transactions that aren't liquidity providing related
            fn_name = tx.functionName.split("(")[0]
            if fn_name not in filter_function:
                continue

            for _log in tx.logs:
                topic, *topic_args = _log["topics"]
                topic_items = topic2sig.get(topic)

                # skip unknown topic name
                if topic_items is None:
                    continue

                topic_name, *topic_params = topic_items

                # filter unwanted topic
                if topic_name not in filter_topics:
                    continue

                # skip blacklist address
                if _log["address"].lower() in list(map(lambda x: x.lower(), blacklist_token)):
                    continue

                # for Transfer
                sender, receiver = topic_args
                sender = hex(int(sender, 16))
                receiver = hex(int(receiver, 16))
                token = Token.address_lookup(_log["address"], chain)
                amount = int(_log["data"], 16) / (10**token.decimal)

                if sender == "0x0":
                    action = "mint"
                    user = receiver
                elif receiver == "0x0":
                    action = "burn"
                    user = sender
                else:
                    # let's focus on mint/burn count
                    # as criteria will be selected 
                    # based on CLP anyway
                    continue

                liquidity_txs.append({
                    "chain": chain,
                    "tx_hash": tx.hash,
                    "sender": sender,
                    "receiver": receiver,
                    "token": token.symbol,
                    "amount": amount,
                    "action": action,
                    "fn_name": fn_name,
                    "user": user,
                    "timestamp": int(tx.timeStamp)
                })

    liquidity_txs = pd.DataFrame(liquidity_txs).sort_values("timestamp")
    liquidity_txs["balance_change"] = liquidity_txs["amount"] * liquidity_txs["action"].map(lambda x: -1 if x == "burn" else 1)
    liquidity_txs["time"] = liquidity_txs["timestamp"].map(unix_to_datetime)
    liquidity_txs = liquidity_txs.drop("timestamp", axis=1).set_index("time").sort_index()
    return liquidity_txs

## Loading Data

In [4]:
api = ConnextAPI(data_dir="data")
data = api.load_cache()

In [5]:
blacklist_token = [
    "0x36955Fb4Ba3618d5a13701f9bb4c2d17436Ca189",  # deprecated polygon CUSDCLP
    "0x9890b51b117f765e9148A12902B0945Fa6d285E5",  # deprecated arbitrum CUSDCLP
    "0xFcc933039AC59F8F16d18B61d99D75fE60A055e3",  # deprecated BNB Chain CUSDCLP
    "0x1AF1b21323dB137603FC9eA8848053647B2C5B37",  # deprecated Gnosis CUSDCLP
    "0x0EB37a910Cb5ac05Ed85C3Be5c2Af5dAf13311B9",  # deprecated optimism CUSDCLP
]
liquidity_txs = get_liquidity_txs(blacklist_token=blacklist_token)

In [10]:
def in_date(time: datetime, date: str) -> bool:
    min_dt = datetime.strptime(date, "%Y-%m-%d")
    max_dt = min_dt + timedelta(hours=24)
    return min_dt <= time < max_dt


def get_daily_txn(df: pd.DataFrame, chain: Chain, date: str) -> pd.DataFrame:
    """Filter transactions for each specific chain and date.
    Date should be in DD-MM-YYYY format"""
    df = df[df["chain"] == chain]
    return df[df.index.map(lambda x: in_date(x, date=date))]


def get_unique_wallets(df: pd.DataFrame) -> Dict[str, List[str]]:
    wallets = {chain: [] for chain in liquidity_txs["chain"].value_counts().index}

    for chain in df["chain"].value_counts().index:
        wallets[chain] = [
            _wallet for _wallet
            in df[df["chain"] == chain]["user"].value_counts().index.tolist()
            if len(_wallet) > 3]
        
    return wallets


def get_top_lp_holders(df: pd.DataFrame, chain: Chain, token: Token) -> pd.Series:
    is_correct_chain = df["chain"] == chain
    is_correct_token = df["token"] == token
    is_mint_and_burn = df["action"].isin(["mint", "burn"])
    df_filter = is_correct_chain & is_correct_token & is_mint_and_burn
    return df[df_filter].groupby(["user"])["balance_change"].sum().sort_values(ascending=False)

In [11]:
dates = list({_item.strftime("%Y-%m-%d") for _item in liquidity_txs.index})
wallets = get_unique_wallets(liquidity_txs)

In [12]:
{_chain: len(addrs) for _chain, addrs in wallets.items()}

{'arbitrum_one': 4061,
 'polygon': 1989,
 'optimism': 921,
 'bnb_chain': 364,
 'gnosis': 119}

Number of unique addresses for each chain

In [29]:
threshold = 0.3
min_value_usdc = 10.
min_value_weth = 0.001

print(f"Applying filter with minimum USDC of {min_value_usdc}")
print(f"Applying filter with minimum WETH of {min_value_weth}\n")

for chain in wallets.keys():
    print(f"Chain: {chain}")
    for token in [Token.CWETHLP, Token.CUSDCLP]:
        df = get_top_lp_holders(liquidity_txs, chain, token)
        n_ori_lp = len(df)
        if token == Token.CUSDCLP:
            df = df[df >= min_value_usdc]
        elif token == Token.CWETHLP:
            df = df[df >= min_value_weth]
        n_filtered_lp = len(df)
        min_value = df.iloc[:round(threshold * len(df))].values[-1]
        print(f"Lowest {token} amount to reach top 30% : {min_value}")
        
        n_removed = n_ori_lp - n_filtered_lp
        print(f"  {n_removed} ({n_removed*100/n_ori_lp:.2f}%) holders are removed for providing less than minimum amount")
    print()

Applying filter with minimum USDC of 10.0
Applying filter with minimum WETH of 0.001

Chain: arbitrum_one
Lowest CWETHLP amount to reach top 30% : 0.08226086952516609
  966 (47.85%) holders are removed for providing less than minimum amount
Lowest CUSDCLP amount to reach top 30% : 299.32295178680386
  1209 (54.73%) holders are removed for providing less than minimum amount

Chain: polygon
Lowest CWETHLP amount to reach top 30% : 0.63730954798902
  29 (34.12%) holders are removed for providing less than minimum amount
Lowest CUSDCLP amount to reach top 30% : 268.92750593583634
  1604 (83.02%) holders are removed for providing less than minimum amount

Chain: optimism
Lowest CWETHLP amount to reach top 30% : 0.5032087896373102
  64 (38.10%) holders are removed for providing less than minimum amount
Lowest CUSDCLP amount to reach top 30% : 269.15846310780194
  535 (65.40%) holders are removed for providing less than minimum amount

Chain: bnb_chain
Lowest CWETHLP amount to reach top 30% :